## 准备数据

In [6]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [7]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [8]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        self.W1 = tf.Variable(tf.random.normal(shape=(28 * 28, 128), stddev=0.01), trainable=True)
        self.b1 = tf.Variable(tf.zeros(shape=(128,)), trainable=True)
        self.W2 = tf.Variable(tf.random.normal(shape=(128, 10), stddev=0.01), trainable=True)
        self.b2 = tf.Variable(tf.zeros(shape=(10,)), trainable=True)
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x_flattened = tf.reshape(x, [-1, 784])
        h1 = tf.matmul(x_flattened, self.W1) + self.b1
        activated_h1 = tf.tanh(h1)
        logits = tf.matmul(activated_h1, self.W2) + self.b2
        return logits

model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [9]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [10]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.3022208 ; accuracy 0.1006
epoch 1 : loss 2.3019662 ; accuracy 0.10598333
epoch 2 : loss 2.3017118 ; accuracy 0.11075
epoch 3 : loss 2.3014574 ; accuracy 0.116283335
epoch 4 : loss 2.3012025 ; accuracy 0.121983334
epoch 5 : loss 2.300948 ; accuracy 0.1278
epoch 6 : loss 2.300693 ; accuracy 0.13378334
epoch 7 : loss 2.300438 ; accuracy 0.13993333
epoch 8 : loss 2.300183 ; accuracy 0.1461
epoch 9 : loss 2.2999282 ; accuracy 0.15168333
epoch 10 : loss 2.299673 ; accuracy 0.15816666
epoch 11 : loss 2.2994177 ; accuracy 0.16493334
epoch 12 : loss 2.2991624 ; accuracy 0.17095
epoch 13 : loss 2.2989068 ; accuracy 0.17701666
epoch 14 : loss 2.2986507 ; accuracy 0.18366666
epoch 15 : loss 2.2983947 ; accuracy 0.18986666
epoch 16 : loss 2.2981384 ; accuracy 0.19703333
epoch 17 : loss 2.2978818 ; accuracy 0.20368333
epoch 18 : loss 2.297625 ; accuracy 0.2094
epoch 19 : loss 2.2973678 ; accuracy 0.2154
epoch 20 : loss 2.2971103 ; accuracy 0.22171667
epoch 21 : loss 2.2968526 ; accu